# Article Page Views API Example
This example illustrates how to access page view data using the [Wikimedia REST API](https://www.mediawiki.org/wiki/Wikimedia_REST_API). This example shows how to request monthly counts of page views for one specific article. The API documentation, [pageviews/per-article](https://wikimedia.org/api/rest_v1/#/Pageviews%20data), covers additional details that may be helpful when trying to use or understand this example.

## License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.2 - August 14, 2023



In [1]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

The example relies on some constants that help make the code a bit more readable.

In [2]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023040100"    # this is likely the wrong end date
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [3]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [4]:
print("Getting pageview data for: ",ARTICLE_TITLES[1])
views = request_pageviews_per_article(ARTICLE_TITLES[1])

Getting pageview data for:  Northern flicker


In [5]:
#print(json.dumps(views,indent=4))
print("Have %d months of pageview data"%(len(views['items'])))
for month in views['items']:
    print(json.dumps(month,indent=4))

Have 93 months of pageview data
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015070100",
    "access": "desktop",
    "agent": "user",
    "views": 4018
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015080100",
    "access": "desktop",
    "agent": "user",
    "views": 3116
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015090100",
    "access": "desktop",
    "agent": "user",
    "views": 4802
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015100100",
    "access": "desktop",
    "agent": "user",
    "views": 5373
}
{
    "project": "en.wikipedia",
    "article": "Northern_flicker",
    "granularity": "monthly",
    "timestamp": "2015110100",
    "access": "desktop",
    "agent": "user",
    "views": 

Above output should show dictionaries with views per month

In [6]:
import pandas as pd
pd.DataFrame(views['items'])

,project,article,granularity,timestamp,access,agent,views
0,en.wikipedia,Northern_flicker,monthly,2015070100,desktop,user,4018
1,en.wikipedia,Northern_flicker,monthly,2015080100,desktop,user,3116
2,en.wikipedia,Northern_flicker,monthly,2015090100,desktop,user,4802
3,en.wikipedia,Northern_flicker,monthly,2015100100,desktop,user,5373
4,en.wikipedia,Northern_flicker,monthly,2015110100,desktop,user,4954
...,...,...,...,...,...,...,...
88,en.wikipedia,Northern_flicker,monthly,2022110100,desktop,user,4114
89,en.wikipedia,Northern_flicker,monthly,2022120100,desktop,user,3728
90,en.wikipedia,Northern_flicker,monthly,2023010100,desktop,user,3492
91,en.wikipedia,Northern_flicker,monthly,2023020100,desktop,user,3211


In [7]:
articles = pd.read_csv('thank_the_academy.AUG.2023.csv.csv')
articles

,name,url
0,Everything Everywhere All at Once,https://en.wikipedia.org/wiki/Everything_Every...
1,All Quiet on the Western Front (2022 film),https://en.wikipedia.org/wiki/All_Quiet_on_the...
2,The Whale (2022 film),https://en.wikipedia.org/wiki/The_Whale_(2022_...
3,Top Gun: Maverick,https://en.wikipedia.org/wiki/Top_Gun:_Maverick
4,Black Panther: Wakanda Forever,https://en.wikipedia.org/wiki/Black_Panther:_W...
...,...,...
1354,The Yankee Doodle Mouse,https://en.wikipedia.org/wiki/The_Yankee_Doodl...
1355,The Yearling (1946 film),https://en.wikipedia.org/wiki/The_Yearling_(19...
1356,"Yesterday, Today and Tomorrow","https://en.wikipedia.org/wiki/Yesterday,_Today..."
1357,You Can't Take It with You (film),https://en.wikipedia.org/wiki/You_Can't_Take_I...


In [8]:
article_list = list(articles['name'])
article_list

['Everything Everywhere All at Once',
 'All Quiet on the Western Front (2022 film)',
 'The Whale (2022 film)',
 'Top Gun: Maverick',
 'Black Panther: Wakanda Forever',
 'Avatar: The Way of Water',
 'Women Talking (film)',
 "Guillermo del Toro's Pinocchio",
 'Navalny (film)',
 'The Elephant Whisperers',
 'An Irish Goodbye',
 'The Boy, the Mole, the Fox and the Horse (film)',
 'RRR (film)',
 'CODA (2021 film)',
 'Dune (2021 film)',
 'The Eyes of Tammy Faye (2021 film)',
 'No Time to Die',
 'The Windshield Wiper',
 'The Long Goodbye (Riz Ahmed album)',
 'The Queen of Basketball',
 'Summer of Soul',
 'Drive My Car (film)',
 'Encanto',
 'West Side Story (2021 film)',
 'Belfast (film)',
 'The Power of the Dog (film)',
 'King Richard (film)',
 'Cruella (film)',
 'Nomadland (film)',
 'The Father (2020 film)',
 'Judas and the Black Messiah',
 'Minari (film)',
 'Mank',
 'Sound of Metal',
 "Ma Rainey's Black Bottom (film)",
 'Promising Young Woman',
 'Tenet (film)',
 'Soul (2020 film)',
 'Another

In [19]:
all_mobile_df = pd.DataFrame()
all_desktop_df = pd.DataFrame()
all_cumulative_df = pd.DataFrame()

for i in range(0,len(article_list)-1):
    subset_views_mobile_web = request_pageviews_per_article(article_list[i], request_template = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-web",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023093000"    # this is likely the wrong end date
})
    
    subset_views_mobile_app = request_pageviews_per_article(article_list[i], request_template = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023093000"    # this is likely the wrong end date
})
    
    subset_views_desktop = request_pageviews_per_article(article_list[i], request_template = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023093000"    # this is likely the wrong end date
})
    
#     print(subset_views)
    try:
        mobile_web_df = pd.DataFrame(subset_views_mobile_web['items'])
        mobile_app_df = pd.DataFrame(subset_views_mobile_app['items'])
        mobile_df = pd.concat([mobile_web_df,mobile_app_df])
        mobile_df = mobile_df.groupby(['project','article','granularity','timestamp','agent'])['views'].sum().reset_index()
        mobile_df['access'] = "mobile"
        all_mobile_df = pd.concat([all_mobile_df,mobile_df])
#         print(all_mobile_df)
        
        desktop_df = pd.DataFrame(subset_views_desktop['items'])
        all_desktop_df = pd.concat([all_desktop_df,desktop_df])
#         print(all_desktop_df)
        
        cumulative_df = pd.concat([mobile_df,desktop_df])
        cumulative_df = cumulative_df.groupby(['project','article','granularity','timestamp','agent'])['views'].sum().reset_index()
        all_cumulative_df = pd.concat([all_cumulative_df,cumulative_df])
#         print(all_cumulative_df)
        
#         all_df = pd.concat([all_df, mobile_df, desktop_df])
        
        if i%100 == 0:
            print(i)
    except:
        print(article_list[i])
    #print(json.dumps(subset_views,indent=4))
    
    
print(all_mobile_df)
all_mobile_df.to_json('academy_monthly_mobile_201501-202309.json', orient = 'split', compression = 'infer', index = 'true')
print(all_desktop_df)
all_desktop_df.to_json('academy_monthly_desktop_201501-202309.json', orient = 'split', compression = 'infer', index = 'true')
print(all_cumulative_df)
all_cumulative_df.to_json('academy_monthly_cumulative_201501-202309.json', orient = 'split', compression = 'infer', index = 'true')

0
100
200
300
400
500
Victor/Victoria
600
700
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/Ryan%27s_Daughter/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D5B0>: Failed to establish a new connection: [WinError 10065] A socket operation was attempted to an unreachable host'))
Ryan's Daughter
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Airport_%281970_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DEE0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikip

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/Interviews_with_My_Lai_Veterans/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DE80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Interviews with My Lai Veterans
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Is_It_Always_Right_to_Be_Right%3F/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D040>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/Is_It_Always_Right_to

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Marooned_%281969_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DF10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/Marooned_%281969_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D670>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/Marooned_%281969_film%29/monthly/2015010100/2023093000 (Caused by New

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/The_Lion_in_Winter_%281968_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D670>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
The Lion in Winter (1968 film)
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Romeo_and_Juliet_%281968_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D760>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/Romeo_and_Juliet_%28

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/Charly/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D370>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Charly
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Journey_into_Self_%281968_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DC70>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/Journey_into_Self_%281968_film%29/monthly/2015010100/2023093000 (Caused

...

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/The_Walls_of_Malapaga/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DAF0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
The Walls of Malapaga
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/The_War_of_the_Worlds_%281953_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/The_War_of_the_Worlds_%281953_film%29

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/The_Wetback_Hound/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D430>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
The Wetback Hound
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/What_Ever_Happened_to_Baby_Jane%3F_%281962_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D5B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/What_Ever_Happened_to_Baby_Jane%

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/Wilson_%281944_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D8B0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/Wilson_%281944_film%29/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DB20>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/Wilson_%281944_film%29/monthly/2015010100/2023093000 (Caused by NewConnec

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/The_Wonderful_World_of_the_Brothers_Grimm/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DD00>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/The_Wonderful_World_of_the_Brothers_Grimm/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DB80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/The_Wonderful_World_of_the_Brothers

HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-web/user/The_Yankee_Doodle_Mouse/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60D430>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/mobile-app/user/The_Yankee_Doodle_Mouse/monthly/2015010100/2023093000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C48F60DAC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
HTTPSConnectionPool(host='wikimedia.org', port=443): Max retries exceeded with url: /api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/desktop/user/The_Yankee_Doodle_Mouse/monthly/2015010100/2023093000 (Caused by NewCon